# Homework 2 Solution

## Part 1: Define Dimensions & Generate Initial Queries

### Identify Key Dimensions

I created key dimensions and example values and saved them in `key_dimensions.yaml`:

In [44]:
key_dimensions = ""
with open('key_dimensions.yaml', 'r') as file:
    key_dimensions = file.read()
print(key_dimensions)

- cuisine_type: [German, Italian, French, Vietnamese]
- meal_type: [breakfast, supper, dinner]
- preparation_time: [less than 1h, 20 min, low prep time]
- available_food: [tuna, salmon, eggs, chicken breast]


### Generate Unique Combinations (Tuples)

I wrote a prompt to generate 15-20 unique combinations (tuples) of the dimension values. Here is the prompt with the key dimension values filled in:

In [3]:
with open('generate-tuples-prompt-tpl.md', 'r') as f:
    template = f.read()

# Interpolate the template with key_dimensions
prompt = template.format(key_dimensions=key_dimensions)

# Print the result to verify
print(prompt)

You are part of a data pipeline to generate evaluations of a recipe chatbot.

Please create 20 unique combinations of following dimension values. The format is `dimension: [value1, value2, ...]`:

- cuisine_type: [German, Italian, French, Vietnamese]
- meal_type: [breakfast, supper, dinner]
- preparation_time: [less than 1h, 20 min, low prep time]
- available_food: [tuna, salmon, eggs, chicken breast]

The combinations shall be realistic for actual user queries and evenly distributed. Use a CSV output format with following columns:

`tuple_id,cuisine_type,meal_type,available_food,preparation_time`

Generate only the csv table with header, no explanations.


Then I prompted `gpt-4.1-nano` to generate the tuples:

In [45]:
import litellm

def call_openai_with_prompt(prompt_text):
    try:
        response = litellm.completion(
            model="gpt-4.1-nano", temperature=0.7, max_tokens=1000,
            messages=[{"role": "user", "content": prompt_text}]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

In [5]:
result = call_openai_with_prompt(prompt)

with open('tuples.csv', 'w', newline='') as csvfile:
    csvfile.write(result)

print("✅ saved to tuples.csv")

✅ saved to tuples.csv


In [6]:
import pandas as pd
display(pd.read_csv('tuples.csv'))

,tuple_id,cuisine_type,meal_type,available_food,preparation_time
0,1,German,breakfast,tuna,less than 1h
1,2,Italian,supper,eggs,20 min
2,3,French,dinner,chicken breast,low prep time
3,4,Vietnamese,breakfast,salmon,less than 1h
4,5,German,dinner,eggs,20 min
5,6,Italian,breakfast,chicken breast,low prep time
6,7,French,supper,tuna,less than 1h
7,8,Vietnamese,dinner,salmon,20 min
8,9,German,supper,eggs,low prep time
9,10,Italian,dinner,tuna,less than 1h


### Generate Natural Language User Queries

I wrote a prompt that asks an LLM to create a natural language user query for each of the tuples. At first I asked it to sample just seven as asked in the homework, but then switched to the whole twenty, to have more data for the error analysis.

In [7]:
tuples = ""
# Read the content of the file as a string
with open('tuples.csv', 'r') as file:
    tuples = file.read()

In [14]:
with open('generate-user-queries-prompt-tpl.md', 'r') as f:
    template = f.read()

# Interpolate the template with tuples
user_queries_prompt = template.format(tuples=tuples)

# Print the result to check them
from IPython.display import Markdown, display
display(Markdown(user_queries_prompt))

You are part of an example data generation pipeline for a recipe chatbot. Your task is to generate realistic user query examples based on following combinations of key dimensions. The key dimensions are stated in the header, the rows contain tuples of unique combinations for the dimensions:

```csv
tuple_id,cuisine_type,meal_type,available_food,preparation_time
1,German,breakfast,tuna,less than 1h
2,Italian,supper,eggs,20 min
3,French,dinner,chicken breast,low prep time
4,Vietnamese,breakfast,salmon,less than 1h
5,German,dinner,eggs,20 min
6,Italian,breakfast,chicken breast,low prep time
7,French,supper,tuna,less than 1h
8,Vietnamese,dinner,salmon,20 min
9,German,supper,eggs,low prep time
10,Italian,dinner,tuna,less than 1h
11,French,breakfast, chicken breast,20 min
12,Vietnamese,supper,eggs,low prep time
13,German,dinner,salmon,less than 1h
14,Italian,breakfast,eggs,20 min
15,French,supper,chicken breast,low prep time
16,Vietnamese,breakfast,tuna,less than 1h
17,German,supper,salmon,20 min
18,Italian,dinner,eggs,low prep time
19,French,breakfast,tuna,less than 1h
20,Vietnamese,breakfast,egg,20 min
```

You can use thinking, but enclose your thinking in <thinking></thinking> delimiters.

For each sample, think about a realistic user persona and write a natural language user query in their voice.

Write the user queries as if they are written by a real human at a smart phone, eg. very short and to the point, no punctuation. Think about different realistic user personas when writing each of the queries but don't include descriptions of the personas in the final result.

Provide your result (and only the result) in following CSV format including headers:

`query_id,tuple_id,user_query`

In [10]:
user_queries_result = call_openai_with_prompt(user_queries_prompt)

with open('user-queries-result.csv', 'w', newline='') as csvfile:
    csvfile.write(user_queries_result)

print("✅ saved to user-queries-result.csv")

✅ saved to user-queries-result.csv


Next, I created a table that merges the tuples with the generated queries, and inspected them to see if some queries need to be improved.

In [11]:
import pandas as pd

# Read the CSVs
user_queries_df = pd.read_csv('user-queries-result.csv')
tuples_df = pd.read_csv('tuples.csv')

# Merge the dataframes on tuple_id
merged_df = pd.merge(user_queries_df, tuples_df, on='tuple_id', how='left')

# Reorder columns for better display
columns_order = ['query_id', 'tuple_id', 'user_query', 'cuisine_type', 'meal_type', 'available_food', 'preparation_time']
merged_df = merged_df[columns_order]

# Display the merged dataframe
display(merged_df)

# Save the merged dataframe to CSV
merged_df.to_csv('user-queries-with-tuples.csv', index=False)
print("✅ Saved to user-queries-with-tuples.csv")

,query_id,tuple_id,user_query,cuisine_type,meal_type,available_food,preparation_time
0,1,1,What can I make for German breakfast with tuna...,German,breakfast,tuna,less than 1h
1,2,2,Quick Italian supper with eggs any ideas,Italian,supper,eggs,20 min
2,3,3,Low prep French dinner chicken breast suggestions,French,dinner,chicken breast,low prep time
3,4,4,Vietnamese breakfast with salmon ready in less...,Vietnamese,breakfast,salmon,less than 1h
4,5,5,Easy German dinner with eggs in 20 minutes,German,dinner,eggs,20 min
5,6,6,Italian breakfast with chicken breast quick re...,Italian,breakfast,chicken breast,low prep time
6,7,7,French supper with tuna under an hour,French,supper,tuna,less than 1h
7,8,8,What to cook Vietnamese dinner with salmon in ...,Vietnamese,dinner,salmon,20 min
8,9,9,German supper with eggs fast options,German,supper,eggs,low prep time
9,10,10,Tuna dinner ideas for Italian in less than an ...,Italian,dinner,tuna,less than 1h


✅ Saved to user-queries-with-tuples.csv


In [13]:
!cat user-queries-with-tuples.csv 

query_id,tuple_id,user_query,cuisine_type,meal_type,available_food,preparation_time
1,1,What can I make for German breakfast with tuna in under an hour,German,breakfast,tuna,less than 1h
2,2,Quick Italian supper with eggs any ideas,Italian,supper,eggs,20 min
3,3,Low prep French dinner chicken breast suggestions,French,dinner,chicken breast,low prep time
4,4,Vietnamese breakfast with salmon ready in less than an hour,Vietnamese,breakfast,salmon,less than 1h
5,5,Easy German dinner with eggs in 20 minutes,German,dinner,eggs,20 min
6,6,Italian breakfast with chicken breast quick recipes,Italian,breakfast,chicken breast,low prep time
7,7,French supper with tuna under an hour,French,supper,tuna,less than 1h
8,8,What to cook Vietnamese dinner with salmon in 20 min,Vietnamese,dinner,salmon,20 min
9,9,German supper with eggs fast options,German,supper,eggs,low prep time
10,10,Tuna dinner ideas for Italian in less than an hour,Italian,dinner,tuna,less than 1h
11,11,French breakfast with chicken 


## Part 2: Initial Error Analysis

### Run Bot on Synthetic Queries

As I wanted to run the queries with the `bulk_test.py` script, I transformed the query results into a format suitable for the bulk script. I also prefixed the id with SYN to distinguish beteen generated, and real user queries.

In [39]:
user_queries_df = pd.read_csv('user-queries-result.csv')
print(user_queries_df)

    query_id  tuple_id                                         user_query
0          1         1  What can I make for German breakfast with tuna...
1          2         2           Quick Italian supper with eggs any ideas
2          3         3  Low prep French dinner chicken breast suggestions
3          4         4  Vietnamese breakfast with salmon ready in less...
4          5         5         Easy German dinner with eggs in 20 minutes
5          6         6  Italian breakfast with chicken breast quick re...
6          7         7              French supper with tuna under an hour
7          8         8  What to cook Vietnamese dinner with salmon in ...
8          9         9               German supper with eggs fast options
9         10        10  Tuna dinner ideas for Italian in less than an ...
10        11        11   French breakfast with chicken breast quick ideas
11        12        12          Vietnamese supper with eggs low prep time
12        13        13     What can I 

In [40]:
# Make a copy of the dataframe to avoid modifying the original
syn_queries_df = user_queries_df.copy()

# Rename query_id column to id and user_query column to query
syn_queries_df = syn_queries_df.rename(columns={'query_id': 'id', 'user_query': 'query'})

# Prefix the id values with SYN
syn_queries_df['id'] = 'SYN' + syn_queries_df['id'].astype(str)

# Drop the tuple_id column
syn_queries_df = syn_queries_df.drop(columns=['tuple_id'])

# Save the data to user-queries.csv
syn_queries_df.to_csv('syn_queries.csv', index=False)

print(f"✅ Saved to user-queries.csv with {len(syn_queries_df)} queries")

✅ Saved to user-queries.csv with 20 queries


In [41]:
!cat syn_queries.csv

id,query
SYN1,What can I make for German breakfast with tuna in under an hour
SYN2,Quick Italian supper with eggs any ideas
SYN3,Low prep French dinner chicken breast suggestions
SYN4,Vietnamese breakfast with salmon ready in less than an hour
SYN5,Easy German dinner with eggs in 20 minutes
SYN6,Italian breakfast with chicken breast quick recipes
SYN7,French supper with tuna under an hour
SYN8,What to cook Vietnamese dinner with salmon in 20 min
SYN9,German supper with eggs fast options
SYN10,Tuna dinner ideas for Italian in less than an hour
SYN11,French breakfast with chicken breast quick ideas
SYN12,Vietnamese supper with eggs low prep time
SYN13,What can I cook German dinner with salmon fast
SYN14,Italian breakfast with eggs quick and easy
SYN15,Low prep French supper with chicken breast suggestions
SYN16,Vietnamese breakfast with tuna ready quickly
SYN17,What to make German supper with salmon in 20 min
SYN18,Italian dinner with eggs low prep quick recipes
SYN19,Quick French breakf

I used the `bulk_test.py` script to query the recipe bot using the synthetic queries.

In [42]:
#!SYSTEM_PROMPT_PATH=systemprompt-003.md \
#    python ../../../scripts/bulk_test.py \
#        --csv syn_queries.csv \
#        --out-path syn_results.csv

In [43]:
!wc syn_results.csv

     831    4815   28458 syn_results.csv


### Open Coding

I reviewed the interaction traces and added descriptive labels/notes to identify patterns and potential errors. I built a tool with Flet (`annotate.py`). I also recored fail/pass (ie. acceptable/not acceptable) to get a quick overview.

![Annotate List View](annotate-list-view.webp)

![Annotate Detail View](annotate-detail-view.webp)

I saw following initial themes, patterns, and potential errors or areas for improvement:

- I found the recipes surprisingly good and interesting.
- ending notes are superflous, should be omitted. This was by far the most prominent failure, I've discovered.
- formatting with preparation is inconsistent (sometimes with bold first words per step, mostly not)
- some cooking time estimations and calorie calculations are off at first look. I assume calory estimations are probably more generally not really correct. Mostly underestimated calories.

### Axial Coding & Taxonomy Definition

I imported the CSV into Excel, and structured the initial open codes into broader, structured categories or 'failure modes' to build an error taxonomy, as described in Sec 3.3 of the provided chapter.

I grouped the observations from open coding into broader categories ("failure modes").

For each identified failure mode, I created a clear and concise taxonomy with
  * **A clear Title** for the failure mode.
  * **A concise one-sentence Definition** explaining the failure mode.
  * **1-2 Illustrative Examples**

In [3]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
from IPython.display import display, HTML

# Read the data from the Excel file, specifically from the 'failure_modes' tab
try:
    failure_modes_df = pd.read_excel('axial-coding.xlsx', sheet_name='failure_modes')
    
    # Display the failure modes data in a formatted way
    display(HTML(failure_modes_df.to_html(index=False)))
    
    # Print some statistics
    print(f"Found {len(failure_modes_df)} failure modes in the axial coding analysis.")
except FileNotFoundError:
    print("Error: The file 'axial-coding.xlsx' was not found.")
except Exception as e:
    print(f"Error reading the Excel file: {str(e)}")

id,title,definition,example1,example2
1,unwanted final note,A final note or question is added but not wanted.,SYN15,SYN19
2,unwanted fat words,"Some words are bold, which is not wanted.",SYN4,NaN
3,unwanted newlines,Superflous newlines appear in lists.,SYN16,NaN
4,wrong calories calculation,The calory calculation is wrong (too low).,SYN9,NaN
5,wrong cook time estimation,The cook time estimation is wrong (too low).,SYN16,NaN


Found 5 failure modes in the axial coding analysis.
